In [ ]:
from state import AgentState
from agents.startup_explorer import startup_explorer_agent
# from agents.legal_risk_analysis import legal_risk_analysis_agent

/Users/sonjiyeong/Documents/GitHub/startup-judge-service/agents/startup_explorer.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model="gpt-4", temperature=0)


In [40]:
import asyncio
import random

async def tech_analysis_agent(company, domain, country):
    await asyncio.sleep(random.uniform(0.5, 1.5))
    return f"[Tech] {company}의 기술 분석 결과입니다."

async def founder_explorer_agent(company, domain, country):
    await asyncio.sleep(random.uniform(0.5, 1.5))
    return f"[Founder] {company} 창업자 평판 분석 완료."

async def market_analysis_agent(company, domain, country):
    await asyncio.sleep(random.uniform(0.5, 1.5))
    return f"[Market] {company}의 시장 분석 결과입니다."

async def competitor_analysis_agent(company, domain, country):
    await asyncio.sleep(random.uniform(0.5, 1.5))
    return f"[Competitor] {company} 경쟁사 분석 결과입니다."

async def legal_risk_analysis_agent(company, domain, country, tech_result):
    await asyncio.sleep(random.uniform(0.5, 1.5))
    return f"[Competitor] {company} 경쟁사 분석 결과입니다."

In [41]:
import asyncio


async def analyze_company(company, domain, country):
    # tech_analysis 백그라운드 실행
    tech_task = asyncio.create_task(tech_analysis_agent(company, domain, country))

    # 나머지 에이전트 동시 실행
    founder_task = asyncio.create_task(founder_explorer_agent(company, domain, country))
    market_task = asyncio.create_task(market_analysis_agent(company, domain, country))
    competitor_task = asyncio.create_task(competitor_analysis_agent(company, domain, country))

    # tech 완료 기다렸다가 legal 시작
    tech_result = await tech_task
    legal_result = await legal_risk_analysis_agent(company, domain, country, tech_result)

    # 나머지 결과 대기
    founder_result = await founder_task
    market_result = await market_task
    competitor_result = await competitor_task

    return {
        "company" : company, 
        "tech_summary": tech_result,
        "founder_reputation": founder_result,
        "market_analysis": market_result,
        "legal_risk": legal_result,
        "competitor_info": competitor_result
    }

In [42]:
result = await analyze_company("뤼튼테크놀로지", "ai chatbot", "global")
result

{'company': '뤼튼테크놀로지',
 'tech_summary': '[Tech] 뤼튼테크놀로지의 기술 분석 결과입니다.',
 'founder_reputation': '[Founder] 뤼튼테크놀로지 창업자 평판 분석 완료.',
 'market_analysis': '[Market] 뤼튼테크놀로지의 시장 분석 결과입니다.',
 'legal_risk': '[Competitor] 뤼튼테크놀로지 경쟁사 분석 결과입니다.',
 'competitor_info': '[Competitor] 뤼튼테크놀로지 경쟁사 분석 결과입니다.'}

In [43]:
state: AgentState = {
    "domain": "AI Chatbot",
    "country": "Global",
    "startup_list": ["뤼튼테크놀로지", "Qure.ai", "Synthesia"],
    "startup_profiles": {},
    "tech_summary": {},
    "founder_reputation": {},
    "market_analysis": {},
    "legal_risk": {},
    "competitor_info": {},
    "investment_decision": {},
    "final_report": {},
}

In [44]:

async def analyze_all_companies(state):
    companies = state["startup_list"]

    tech_summary = {}
    founder_reputation = {}
    market_analysis = {}
    legal_risk = {}
    competitor_info = {}

    tasks = [
        analyze_company(
            company, state["domain"], state["country"]
        ) for company in companies
    ]

    company_results = await asyncio.gather(*tasks)

    # 결과 취합
    for i, company in enumerate(companies):
        tech_summary[company] = company_results[i]["tech_summary"]
        founder_reputation[company] = company_results[i]["founder_reputation"]
        market_analysis[company] = company_results[i]["market_analysis"]
        legal_risk[company] = company_results[i]["legal_risk"]
        competitor_info[company] = company_results[i]["competitor_info"]

    return {
        "tech_summary": tech_summary,
        "founder_reputation": founder_reputation, 
        "market_analysis" : market_analysis, 
        "legal_risk" : legal_risk, 
        "competitor_info" : competitor_info, 
    }

